In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import copy
from tqdm.notebook import tqdm_notebook
import math
from skopt import gbrt_minimize
import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

random_state = 1

mps


In [2]:
# read in data and split
X = torch.load('inputs/rfw_senet50_face_embeddings.pt').cpu()
y = torch.load('inputs/rfw_senet50_labels.pt').cpu()
df = pd.read_csv('inputs/rfw_senet50_df.csv')
df['reference_ethnicity'] = df['reference_ethnicity'].str.lower()
ethnicity = df['reference_ethnicity']
df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
0,m.0c7mh2,m.0c7mh2,african,African,1.0
1,m.0c7mh2,m.0c7mh2,african,African,1.0
2,m.026tq86,m.026tq86,african,African,1.0
3,m.026tq86,m.026tq86,african,African,1.0
4,m.02wz3nc,m.02wz3nc,african,African,1.0
...,...,...,...,...,...
29311,m.0402tg,m.01npnk3,caucasian,Caucasian,0.0
29312,m.05pbbnj,m.02rrb2n,caucasian,Caucasian,0.0
29313,m.09j6df,m.07kcsqd,african,African,0.0
29314,m.0fhrbz,m.025zgjt,african,African,0.0


In [3]:
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)

train_X = X[train_split]
X_val = X[val_split]
test_X = X[test_split]

train_y = y[train_split]
y_val = y[val_split]
test_y = y[test_split]

train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]

ethnicity = df['reference_ethnicity'].values
ethnicity[ethnicity=='caucasian'] = 0
ethnicity[ethnicity=='african'] = 1
ethnicity = ethnicity.astype(int)

train_ethnicity = ethnicity[train_split]


ethnicity_val = ethnicity[val_split]

test_ethnicity = ethnicity[test_split]

In [4]:
train_matches = train_df[train_df.labels==1]
train_non_matches = train_df[train_df.labels==0]
print('non matches')
print(train_non_matches['reference_ethnicity'].value_counts())
print('matches')
print(train_matches['reference_ethnicity'].value_counts())

african_non_matches = train_non_matches[train_non_matches['reference_ethnicity']=='african']
caucasian_non_matches = train_non_matches[train_non_matches['reference_ethnicity']=='caucasian']

african_matches = train_matches[train_matches['reference_ethnicity']=='african']
caucasian_matches = train_matches[train_matches['reference_ethnicity']=='caucasian']

non matches
african      4492
caucasian    4376
Name: reference_ethnicity, dtype: int64
matches
african      4396
caucasian    4325
Name: reference_ethnicity, dtype: int64


In [5]:
np.random.seed(random_state)
african_matches_sub_idx = african_matches.index[np.random.choice(len(african_matches.index), size=4325, replace=False)]
caucasian_non_matches_sub_idx = caucasian_non_matches.index[np.random.choice(len(caucasian_non_matches.index), size=4325, replace=False)]
african_non_matches_sub_idx = african_non_matches.index[np.random.choice(len(african_non_matches.index), size=4325, replace=False)]
caucasian_matches_sub_idx = caucasian_matches.index

X_train = torch.cat([X[african_matches_sub_idx],X[caucasian_matches_sub_idx],X[african_non_matches_sub_idx],X[caucasian_non_matches_sub_idx]])
y_train = torch.cat([y[african_matches_sub_idx],y[caucasian_matches_sub_idx],y[african_non_matches_sub_idx],y[caucasian_non_matches_sub_idx]])
ethnicity_train = np.concatenate([ethnicity[african_matches_sub_idx],ethnicity[caucasian_matches_sub_idx],ethnicity[african_non_matches_sub_idx],ethnicity[caucasian_non_matches_sub_idx]])
df_train = pd.concat([df.iloc[african_matches_sub_idx],df.iloc[caucasian_matches_sub_idx],df.iloc[african_non_matches_sub_idx],df.iloc[caucasian_non_matches_sub_idx]])
df_train

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
6419,m.03j6s0,m.03j6s0,1,African,1.0
5541,m.01qd5bm,m.01qd5bm,1,African,1.0
5881,m.02qdp4v,m.02qdp4v,1,African,1.0
229,m.0272001,m.0272001,1,African,1.0
5264,m.05f962,m.05f962,1,African,1.0
...,...,...,...,...,...
18810,m.0g3f1p,m.01kxfvq,0,Caucasian,0.0
20550,m.07gb0p,m.0bgwxt,0,Caucasian,0.0
24018,m.07pyvy,m.01y7m1,0,Caucasian,0.0
20879,m.02qg85,m.070p26,0,Caucasian,0.0


In [6]:
test_split, holdout_split = train_test_split(np.arange(5864),test_size=0.1, random_state=random_state)
X_test = test_X[test_split]
X_holdout = test_X[holdout_split]
y_test = test_y[test_split]
y_holdout = test_y[holdout_split]
ethnicity_test = test_ethnicity[test_split]
ethnicity_holdout = test_ethnicity[holdout_split]
df_test = test_df.iloc[test_split]
df_holdout = test_df.iloc[holdout_split]
df_holdout

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
26599,m.08dyjb,m.01dysq,african,African,0.0
13208,m.05nqz8,m.05nqz8,caucasian,Caucasian,1.0
7459,m.0chlpf,m.0chlpf,caucasian,Caucasian,1.0
4747,m.0gzkxh,m.0gzkxh,african,African,1.0
22098,m.01tl0vh,m.027g5hs,african,African,0.0
...,...,...,...,...,...
27224,m.0glqs_8,m.05972y,african,African,0.0
5728,m.06w9s22,m.06w9s22,african,African,1.0
18206,m.04zvsy6,m.06tvw7,caucasian,Caucasian,0.0
25909,m.0h_dpyh,m.04d_dnm,african,African,0.0


In [7]:
## train data
class TrainData(data.Dataset):
    
    def __init__(self, X_data, y_data,ethnicity):
        self.X_data = X_data
        self.y_data = y_data
        self.ethnicity = ethnicity    
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index], self.ethnicity[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train),
                       ethnicity_train)
test_data = TrainData(torch.FloatTensor(X_test),torch.FloatTensor(y_test),ethnicity_test)
holdout_data = TrainData(torch.FloatTensor(X_holdout),torch.FloatTensor(y_holdout),ethnicity_holdout)
val_data = TrainData(torch.FloatTensor(X_val), 
                       torch.FloatTensor(y_val),
                       ethnicity_val)
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 4096
        self.layer_1 = nn.Linear(4096, 1024) 
        self.layer_2 = nn.Linear(1024, 512)
        self.layer_out = nn.Linear(512, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(1024)
        self.batchnorm2 = nn.BatchNorm1d(512)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
    


In [8]:
EPOCHS = 20
BATCH_SIZE = 64
LEARNING_RATE = 1e-5

model = torch.load('weights/rfw_senet50_logistic_regression_face_matching_0.0_betaTEST.pt')
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_loader = data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
holdout_loader = data.DataLoader(dataset=holdout_data, batch_size=BATCH_SIZE)
val_loader = data.DataLoader(dataset=val_data, batch_size=BATCH_SIZE)


In [9]:
def val_model(model, loader, criterion):
    """Validate model on loader with criterion function"""
    y_true, y_pred, y_prot = [], [], []
    model.eval()
    with torch.no_grad():
        for inputs, labels,protected in loader:
            inputs = inputs.to(device)
            y_true.append(labels)
            y_prot.append(protected)
            y_pred.append(torch.sigmoid(model(inputs)).cpu())
    y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
    return criterion(y_true, y_pred, y_prot)
def compute_bias(y_pred, y_true, prot, metric):
    """Compute bias on the dataset"""
    def zero_if_nan(data):
        """Zero if there is a nan"""
        return 0. if torch.isnan(data) else data

    gtpr_prot = zero_if_nan(y_pred[prot * y_true == 1].mean())
    gfpr_prot = zero_if_nan(y_pred[prot * (1-y_true) == 1].mean())
    mean_prot = zero_if_nan(y_pred[prot == 1].mean())

    gtpr_unprot = zero_if_nan(y_pred[(1-prot) * y_true == 1].mean())
    gfpr_unprot = zero_if_nan(y_pred[(1-prot) * (1-y_true) == 1].mean())
    mean_unprot = zero_if_nan(y_pred[(1-prot) == 1].mean())

    if metric == "spd":
        return mean_prot - mean_unprot
    elif metric == "aod":
        return 0.5 * ((gfpr_prot - gfpr_unprot) + (gtpr_prot - gtpr_unprot))
    elif metric == "eod":
        return gtpr_prot - gtpr_unprot
    elif 'false_diff':
        return (np.abs(gfpr_prot - gfpr_unprot) + np.abs(gtpr_unprot - gtpr_prot))

def compute_objective(performance, bias, epsilon=0.01, margin=0.005):
    if abs(bias) <= (epsilon-margin):
        return performance
    else:
        return 0.0
def get_best_objective(y_true, y_pred, y_prot):
    """Find the threshold for the best objective"""
    num_samples = 5
    threshs = torch.linspace(0, 1, 501)
    best_obj, best_thresh = -math.inf, 0.
    for thresh in threshs:
        indices = np.random.choice(np.arange(y_pred.size()[0]), num_samples*y_pred.size()[0], replace=True).reshape(num_samples, y_pred.size()[0])
        objs = []
        for index in indices:
            y_pred_tmp = y_pred[index]
            y_true_tmp = y_true[index]
            y_prot_tmp = y_prot[index]
            perf = (torch.mean((y_pred_tmp > thresh)[y_true_tmp.type(torch.bool)].type(torch.float32)) + torch.mean((y_pred_tmp <= thresh)[~y_true_tmp.type(torch.bool)].type(torch.float32))) / 2
            bias = compute_bias((y_pred_tmp > thresh).float().cpu(), y_true_tmp.float().cpu(), y_prot_tmp.float().cpu(), 'aod')
            objs.append(compute_objective(perf, bias))
        obj = float(torch.tensor(objs).mean())
        if obj > best_obj:
            best_obj, best_thresh = obj, thresh

    return best_obj, best_thresh

In [10]:
# compute bias before applying random processing
y_true, y_pred, y_prot = [], [], []
model.eval()
with torch.no_grad():
    for inputs, labels,protected in val_loader:
        inputs = inputs.to(device)
        y_true.append(labels)
        y_prot.append(protected)
        y_pred.append(torch.sigmoid(model(inputs)).cpu())
y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)

print(compute_bias((y_pred>0.5).float().cpu(), y_true, y_prot, 'false_diff'))
print(compute_bias((y_pred>0.5).float().cpu(), y_true, y_prot, 'aod'))

tensor(0.0197)
tensor(0.0015)


In [11]:
num_iterations = 100
torch.manual_seed(random_state)
np.random.seed(random_state)
rand_result = [-np.inf, None, -1]

for iteration in tqdm_notebook(range(num_iterations),total=num_iterations):
    rand_model = copy.deepcopy(model)
    rand_model = rand_model.to(device)
    for param in rand_model.parameters():
        param.data = param.data * (torch.randn_like(param) * 0.1 + 1)

    rand_model.eval()
    best_obj, best_thresh = val_model(rand_model, test_loader, get_best_objective)

    if best_obj > rand_result[0]:

        del rand_result[1]
        rand_result = [best_obj, copy.deepcopy(rand_model.state_dict()), best_thresh]

# evaluate best random model
best_model = copy.deepcopy(model)
best_model.load_state_dict(rand_result[1])
best_model.to(device)
best_thresh = rand_result[2]


  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:

y_true, y_pred, y_prot = [], [], []
best_model.eval()
with torch.no_grad():
    for inputs, labels,protected in test_loader:
        inputs = inputs.to(device)
        y_true.append(labels)
        y_prot.append(protected)
        y_pred.append(torch.sigmoid(best_model(inputs)).cpu())
y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'false_diff')
# torch.save(best_model.state_dict(), 'seed'+str(random_state)+'aod'+config['optimizer']+)

tensor(0.0264)

In [13]:
# evaluate bias on holdout set after mitigation
y_true, y_pred, y_prot = [], [], []
best_model.eval()
with torch.no_grad():
    for inputs, labels,protected in holdout_loader:
        inputs = inputs.to(device)
        y_true.append(labels)
        y_prot.append(protected)
        y_pred.append(torch.sigmoid(best_model(inputs)).cpu())
y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
print(compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'false_diff'))
print(compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'aod'))

y_unpriv_ids = (y_prot==1)
y_unpriv_pred = y_pred[y_unpriv_ids] > best_thresh
y_unpriv_label = y_true[y_unpriv_ids]
y_priv_ids = (y_prot==0)
y_priv_pred = y_pred[y_priv_ids]> best_thresh
y_priv_label = y_true[y_priv_ids]



y_priv_tn, y_priv_fp, y_priv_fn, y_priv_tp, y_priv_acc = confusion_mat(y_priv_pred, y_priv_label)
y_priv_fpr = y_priv_fp/(y_priv_fp+y_priv_tn)
y_priv_fnr = y_priv_fn/(y_priv_fn+y_priv_tp)

y_unpriv_tn, y_unpriv_fp, y_unpriv_fn, y_unpriv_tp, y_unpriv_acc = confusion_mat(y_unpriv_pred, y_unpriv_label)
y_unpriv_fpr = y_unpriv_fp/(y_unpriv_fp+y_unpriv_tn)
y_unpriv_fnr = y_unpriv_fn/(y_unpriv_fn+y_unpriv_tp)


print('Missclassificaition Rate Ratio:',round((1-y_unpriv_acc)/(1- y_priv_acc),3))
print('FPR Ratio:',y_unpriv_fpr/y_priv_fpr)
print('FNR Ratio:',y_unpriv_fnr/y_priv_fnr)

print('AOD:',compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'aod'))
print('False Diff:',compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'false_diff'))
print('Best Threshold:', best_thresh)
confusion_mat((y_pred>best_thresh).float().cpu(), y_true)

tensor(0.1186)
tensor(-0.0593)
Missclassificaition Rate Ratio: 1.112
FPR Ratio: 0.5832642916321458
FNR Ratio: 2.910843373493976
AOD: tensor(-0.0593)
False Diff: tensor(0.1186)
Best Threshold: tensor(0.6820)


(242, 28, 21, 296, 0.9165247018739353)

In [14]:
print('tnm', 'fm', 'fnm', 'tm')
print(confusion_mat(y_unpriv_pred, y_unpriv_label))
print(confusion_mat(y_priv_pred, y_priv_label))
print(y_unpriv_fpr,y_unpriv_fnr)
print(y_priv_fpr,y_priv_fnr)
confusion_mat((y_pred>best_thresh).float().cpu(), y_true)

tnm fm fnm tm
(131, 11, 16, 150, 0.9123376623376623)
(111, 17, 5, 146, 0.921146953405018)
0.07746478873239436 0.0963855421686747
0.1328125 0.033112582781456956


(242, 28, 21, 296, 0.9165247018739353)

In [14]:
torch.save(best_model.state_dict(), 'weights/rfw_senet_random_intraprocessing_best_model.pt')

In [15]:
confusion_mat((y_pred>best_thresh).float().cpu(), y_true)

(242, 28, 21, 296, 0.9165247018739353)